# Ground truth for fuglene

In [1]:
'''
9286 158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0011_159736.jpg
9286 65.0 92.0 353.0 213.0

9273 158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0100_797142.jpg
9273 202.0 116.0 256.0 180.0

10840 184.Louisiana_Waterthrush/Louisiana_Waterthrush_0022_177397.jpg
10840 85.0 85.0 253.0 176.0

3344 058.Pigeon_Guillemot/Pigeon_Guillemot_0043_39861.jpg
3344 75.0 61.0 244.0 278.0

3350 058.Pigeon_Guillemot/Pigeon_Guillemot_0109_39872.jpg
3350 71.0 15.0 139.0 181.0

11068 188.Pileated_Woodpecker/Pileated_Woodpecker_0027_179956.jpg
11068 76.0 31.0 320.0 467.0

7446 127.Savannah_Sparrow/Savannah_Sparrow_0093_118267.jpg
7446 163.0 95.0 133.0 278.0

10531 179.Tennessee_Warbler/Tennessee_Warbler_0022_174799.jpg
10531 78.0 119.0 246.0 153.0

10512 179.Tennessee_Warbler/Tennessee_Warbler_0067_174999.jpg
10512 122.0 127.0 255.0 217.0

11691 199.Winter_Wren/Winter_Wren_0113_189558.jpg
11691 185.0 32.0 208.0 258.0
'''

'\n9286 158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0011_159736.jpg\n9286 65.0 92.0 353.0 213.0\n\n9273 158.Bay_breasted_Warbler/Bay_Breasted_Warbler_0100_797142.jpg\n9273 202.0 116.0 256.0 180.0\n\n10840 184.Louisiana_Waterthrush/Louisiana_Waterthrush_0022_177397.jpg\n10840 85.0 85.0 253.0 176.0\n\n3344 058.Pigeon_Guillemot/Pigeon_Guillemot_0043_39861.jpg\n3344 75.0 61.0 244.0 278.0\n\n3350 058.Pigeon_Guillemot/Pigeon_Guillemot_0109_39872.jpg\n3350 71.0 15.0 139.0 181.0\n\n11068 188.Pileated_Woodpecker/Pileated_Woodpecker_0027_179956.jpg\n11068 76.0 31.0 320.0 467.0\n\n7446 127.Savannah_Sparrow/Savannah_Sparrow_0093_118267.jpg\n7446 163.0 95.0 133.0 278.0\n\n10531 179.Tennessee_Warbler/Tennessee_Warbler_0022_174799.jpg\n10531 78.0 119.0 246.0 153.0\n\n10512 179.Tennessee_Warbler/Tennessee_Warbler_0067_174999.jpg\n10512 122.0 127.0 255.0 217.0\n\n11691 199.Winter_Wren/Winter_Wren_0113_189558.jpg\n11691 185.0 32.0 208.0 258.0\n'